# Model to Predict Elo Customer Loyalty

_Note! If you want to commit any changes to this document, please strip all output (Cell > Current Outputs > Clear, or set up [nbstripout](https://github.com/kynan/nbstripout) as a git filter) from this notebook before doing so. Thanks!_


## Import Libraries

Next we import the Python libraries we'll need. If any of these are missing for you, you can install them with e.g. `pip3 install pandas` on the command line.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load Data

Load the data into Pandas data frames and look at their structure.

First thing we'll do with the training data is split it into a train and validation set. (The given test set is what we'll later make our predictions on and upload, but only after we are fully satisfied with our model.)

**Make sure to run `make processdata` (which takes a very long time, but only needs to be done once) before running the code in this notebook!**

In [ ]:
train_and_validation_df = pd.read_csv('data/processed/train_with_aggregated_features.csv', index_col='card_id')
test_df = pd.read_csv('data/processed/test_with_aggregated_features.csv', index_col='card_id')

In [ ]:
train_and_validation_df.head()

## Split Into Train and Validation Sets

Split our data into a train test (80%) and a validation set (20%).

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(train_and_validation_df, test_size=0.2, random_state=238923)

In [ ]:
train_df.shape

In [ ]:
validate_df.shape

In [ ]:
train_df.head()

## Remove Outliers

We shouldn't actually ever do this manually, except for experimental purposes. Spoiler: the outliers have a large impact on the final performance of our model.

In [ ]:
# train_df = train_df[train_df.target > -25]

## A Quick Look at Correlations

In [ ]:
train_df.corr().target.sort_values(ascending=False)

## Set Up Model

We'll use the fastai tabular regressor here, which is built for exactly this sort of problem.

In [ ]:
from fastai import *
from fastai.tabular import *
from fastai.metrics import *

### Create Data Bunch

A fastai DataBunch more or less contains the data that we'll feed to our model.

First, as the data bunch takes one data frame containing both the test and validation samples, we need to get the indices for our validation samples.

Then we tell the model which of the columns are categorical features, which are continuous features, and also which of the columns contains the target (the value we want to predict).

In [ ]:
valid_idx = range(len(train_df), len(train_df) + len(validate_df)); valid_idx

Let's have a look at which columns we have. We will need to tell fastai which ones are categorical and which ones are continuous.

In [ ]:
for c in train_df.columns: print(c)

In [ ]:
category_names = ['first_active_monthYear',
                  'first_active_monthMonth',
                  'first_active_monthWeek',
                  'first_active_monthIs_quarter_start',
                  'first_active_monthIs_year_start',
                  'feature_1',
                  'feature_2',
                  'feature_3',
                  'authorized_flag_top',
                  'category_1_transaction_top',
                  'category_1_merchant_top',
                  'category_2_top',
                  'category_3_top',
                  'category_4_top',
                  'subsector_id_transaction_top',
                  'subsector_id_merchant_top',
                  'city_id_top',
                  'state_id_top',
                  'purchase_Year_top',
                  'purchase_Month_top',
                  'purchase_Week_top',
                  'purchase_Day_top',
                  'purchase_Dayofweek_top',
                  'most_recent_sales_range_top',
                  'most_recent_purchases_range_top',
                  'merch_authorized_flag_top',
                  'merch_category_1_transaction_top',
                  'merch_category_1_merchant_top',
                  'merch_category_2_top',
                  'merch_category_3_top',
                  'merch_category_4_top',
                  'merch_subsector_id_transaction_top',
                  'merch_subsector_id_merchant_top',
                  'merch_city_id_top',
                  'merch_state_id_top',
                  'merch_purchase_Year_top',
                  'merch_purchase_Month_top',
                  'merch_purchase_Week_top',
                  'merch_purchase_Day_top',
                  'merch_purchase_Dayofweek_top',
                  'merch_most_recent_sales_range_top',
                  'merch_most_recent_purchases_range_top',]
continuous_names = ['first_active_monthElapsed',
                    'purchase_amount_sum',
                    'purchase_amount_mean',
                    'purchase_amount_min',
                    'purchase_amount_max',
                    'purchase_amount_std',
                    'installments_sum',
                    'installments_mean',
                    'installments_min',
                    'installments_max',
                    'installments_std',
                    'month_lag_mean',
                    'month_lag_min',
                    'month_lag_max',
                    'merchant_id_nunique',
                    'state_id_nunique',
                    'city_id_nunique',
                    'numerical_1_sum',
                    'numerical_1_mean',
                    'numerical_1_min',
                    'numerical_1_max',
                    'numerical_1_std',
                    'numerical_2_sum',
                    'numerical_2_mean',
                    'numerical_2_min',
                    'numerical_2_max',
                    'numerical_2_std',
                    'avg_sales_lag3_sum',
                    'avg_sales_lag3_mean',
                    'avg_sales_lag3_min',
                    'avg_sales_lag3_max',
                    'avg_sales_lag3_std',
                    'avg_sales_lag6_sum',
                    'avg_sales_lag6_mean',
                    'avg_sales_lag6_min',
                    'avg_sales_lag6_max',
                    'avg_sales_lag6_std',
                    'avg_sales_lag12_sum',
                    'avg_sales_lag12_mean',
                    'avg_sales_lag12_min',
                    'avg_sales_lag12_max',
                    'avg_sales_lag12_std',
                    'avg_purchases_lag3_sum',
                    'avg_purchases_lag3_mean',
                    'avg_purchases_lag3_min',
                    'avg_purchases_lag3_max',
                    'avg_purchases_lag3_std',
                    'avg_purchases_lag6_sum',
                    'avg_purchases_lag6_mean',
                    'avg_purchases_lag6_min',
                    'avg_purchases_lag6_max',
                    'avg_purchases_lag6_std',
                    'avg_purchases_lag12_sum',
                    'avg_purchases_lag12_mean',
                    'avg_purchases_lag12_min',
                    'avg_purchases_lag12_max',
                    'avg_purchases_lag12_std',
                    'active_months_lag3_sum',
                    'active_months_lag3_mean',
                    'active_months_lag3_min',
                    'active_months_lag3_std',
                    'active_months_lag6_sum',
                    'active_months_lag6_mean',
                    'active_months_lag6_min',
                    'active_months_lag6_std',
                    'active_months_lag12_sum',
                    'active_months_lag12_mean',
                    'active_months_lag12_min',
                    'active_months_lag12_max',
                    'active_months_lag12_std',
                    'merchant_category_id_transaction_nunique',
                    'merchant_category_id_merchant_nunique',
                    'subsector_id_transaction_nunique',
                    'subsector_id_merchant_nunique',
                    'merchant_group_id_nunique',
                    'most_recent_sales_range_nunique',
                    'most_recent_purchases_range_nunique',
                    'elapsed_since_last_purchase_sum',
                    'elapsed_since_last_purchase_mean',
                    'elapsed_since_last_purchase_min',
                    'elapsed_since_last_purchase_max',
                    'elapsed_since_last_purchase_std',
                    'elapsed_since_last_merch_purchase_sum',
                    'elapsed_since_last_merch_purchase_mean',
                    'elapsed_since_last_merch_purchase_min',
                    'elapsed_since_last_merch_purchase_max',
                    'elapsed_since_last_merch_purchase_std',
                    'authorized_flag_Y_ratio',
                    'category_1_transaction_N_ratio',
                    'category_1_merchant_N_ratio',
                    'category_2_1.0_ratio',
                    'category_2_3.0_ratio',
                    'category_2_4.0_ratio',
                    'category_2_2.0_ratio',
                    'category_2_5.0_ratio',
                    'category_3_A_ratio',
                    'category_3_B_ratio',
                    'category_3_C_ratio',
                    'category_4_N_ratio',
                    'purchase_Is_month_start_True_ratio',
                    'purchase_Is_month_end_False_ratio',
                    'purchase_Year_2017_ratio',
                    'most_recent_sales_range_B_ratio',
                    'most_recent_sales_range_A_ratio',
                    'most_recent_sales_range_C_ratio',
                    'most_recent_sales_range_D_ratio',
                    'most_recent_sales_range_E_ratio',
                    'most_recent_purchases_range_B_ratio',
                    'most_recent_purchases_range_C_ratio',
                    'most_recent_purchases_range_A_ratio',
                    'most_recent_purchases_range_D_ratio',
                    'most_recent_purchases_range_E_ratio',
                    'merch_purchase_amount_sum',
                    'merch_purchase_amount_mean',
                    'merch_purchase_amount_min',
                    'merch_purchase_amount_max',
                    'merch_purchase_amount_std',
                    'merch_installments_sum',
                    'merch_installments_mean',
                    'merch_installments_min',
                    'merch_installments_max',
                    'merch_installments_std',
                    'merch_month_lag_mean',
                    'merch_month_lag_min',
                    'merch_month_lag_max',
                    'merch_merchant_id_nunique',
                    'merch_state_id_nunique',
                    'merch_city_id_nunique',
                    'merch_numerical_1_sum',
                    'merch_numerical_1_mean',
                    'merch_numerical_1_min',
                    'merch_numerical_1_max',
                    'merch_numerical_1_std',
                    'merch_numerical_2_sum',
                    'merch_numerical_2_mean',
                    'merch_numerical_2_min',
                    'merch_numerical_2_max',
                    'merch_numerical_2_std',
                    'merch_avg_sales_lag3_sum',
                    'merch_avg_sales_lag3_mean',
                    'merch_avg_sales_lag3_min',
                    'merch_avg_sales_lag3_max',
                    'merch_avg_sales_lag3_std',
                    'merch_avg_sales_lag6_sum',
                    'merch_avg_sales_lag6_mean',
                    'merch_avg_sales_lag6_min',
                    'merch_avg_sales_lag6_max',
                    'merch_avg_sales_lag6_std',
                    'merch_avg_sales_lag12_sum',
                    'merch_avg_sales_lag12_mean',
                    'merch_avg_sales_lag12_min',
                    'merch_avg_sales_lag12_max',
                    'merch_avg_sales_lag12_std',
                    'merch_avg_purchases_lag3_sum',
                    'merch_avg_purchases_lag3_mean',
                    'merch_avg_purchases_lag3_min',
                    'merch_avg_purchases_lag3_max',
                    'merch_avg_purchases_lag3_std',
                    'merch_avg_purchases_lag6_sum',
                    'merch_avg_purchases_lag6_mean',
                    'merch_avg_purchases_lag6_min',
                    'merch_avg_purchases_lag6_max',
                    'merch_avg_purchases_lag6_std',
                    'merch_avg_purchases_lag12_sum',
                    'merch_avg_purchases_lag12_mean',
                    'merch_avg_purchases_lag12_min',
                    'merch_avg_purchases_lag12_max',
                    'merch_avg_purchases_lag12_std',
                    'merch_active_months_lag3_sum',
                    'merch_active_months_lag3_mean',
                    'merch_active_months_lag3_min',
                    'merch_active_months_lag3_max',
                    'merch_active_months_lag3_std',
                    'merch_active_months_lag6_sum',
                    'merch_active_months_lag6_mean',
                    'merch_active_months_lag6_min',
                    'merch_active_months_lag6_max',
                    'merch_active_months_lag6_std',
                    'merch_active_months_lag12_sum',
                    'merch_active_months_lag12_mean',
                    'merch_active_months_lag12_min',
                    'merch_active_months_lag12_max',
                    'merch_active_months_lag12_std',
                    'merch_merchant_category_id_transaction_nunique',
                    'merch_merchant_category_id_merchant_nunique',
                    'merch_subsector_id_transaction_nunique',
                    'merch_subsector_id_merchant_nunique',
                    'merch_merchant_group_id_nunique',
                    'merch_most_recent_sales_range_nunique',
                    'merch_most_recent_purchases_range_nunique',
                    'merch_elapsed_since_last_purchase_sum',
                    'merch_elapsed_since_last_purchase_mean',
                    'merch_elapsed_since_last_purchase_min',
                    'merch_elapsed_since_last_purchase_max',
                    'merch_elapsed_since_last_purchase_std',
                    'merch_category_1_transaction_N_ratio',
                    'merch_category_1_merchant_N_ratio',
                    'merch_category_2_1.0_ratio',
                    'merch_category_2_4.0_ratio',
                    'merch_category_2_2.0_ratio',
                    'merch_category_2_3.0_ratio',
                    'merch_category_2_5.0_ratio',
                    'merch_category_3_B_ratio',
                    'merch_category_3_A_ratio',
                    'merch_category_3_C_ratio',
                    'merch_category_4_Y_ratio',
                    'merch_category_4_N_ratio',
                    'merch_purchase_Is_month_start_True_ratio',
                    'merch_purchase_Is_month_end_False_ratio',
                    'merch_purchase_Year_2017_ratio',
                    'merch_purchase_Year_2018_ratio',
                    'merch_most_recent_sales_range_A_ratio',
                    'merch_most_recent_sales_range_B_ratio',
                    'merch_most_recent_sales_range_D_ratio',
                    'merch_most_recent_sales_range_E_ratio',
                    'merch_most_recent_sales_range_C_ratio',
                    'merch_most_recent_purchases_range_A_ratio',
                    'merch_most_recent_purchases_range_B_ratio',
                    'merch_most_recent_purchases_range_D_ratio',
                    'merch_most_recent_purchases_range_C_ratio',
                    'merch_most_recent_purchases_range_E_ratio',]
dep_var = 'target'

Since we picked our validation samples randomly from the initial data set, and since fastai requires us to give the indices of the validation samples in a data frame containing both the training and validation samples, we just concatenate them together with training samples first and the validation samples at the end.

In [ ]:
df = pd.concat([train_df, validate_df]).reset_index()[category_names + continuous_names + [dep_var]]

In [ ]:
data = (TabularList.from_df(df,
                            path='learner',
                            cat_names=category_names,
                            cont_names=continuous_names,
                            procs=[FillMissing, Categorify, Normalize])
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList)
                .databunch())

Let's have a look at a random batch of data to see how it looks after the processing done by the fastai library.

In [ ]:
data.show_batch()

### Create Learner

This is what we actually use to train the model and make predictions.

First we decide how large we want to make the embeddings of our categorical features (the number of category options divided by 2 is a good heuristic, apparently).

Then we tell the model the range within which we expect all predictions to fall (internally the model uses a sigmoid function, so in order for us, in practice, to actually get predictions near the expected maximum value, we set the upper bound to be a little higher than the expected maximum).

The competition uses root mean squared error to evaluate the entries, so we'll use that, too.

In [ ]:
min_y = np.min(train_df['target']) * 1.2
max_y = np.max(train_df['target']) * 1.2
y_range = torch.tensor([min_y, max_y], device=defaults.device); y_range

In [ ]:
np.min(train_df['target']), np.max(train_df['target'])

In [ ]:
learn = tabular_learner(data,
                        layers=[400, 200],
                        ps=[2e-1, 1e-1],
                        emb_drop=0.04,
                        y_range=y_range,
                        metrics=rmse)

In [ ]:
learn.model

We can also load an already trained model (look into `learner/models/`) like so:

In [ ]:
# learn.load('model_cv_3_908')

### Figure Out Learning Rate

To figure out which learning rate to use, we use fastai's learning rate finder.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Train Model

Finally we train the model, with weight decay to encourage the model to use fewer features, and then show some results.

In [ ]:
learn.fit_one_cycle(3, 1e-3, wd=0.7)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.show_results()

## Make Predictions

Now that we have trained our model, lets make some predictions to see whether or not our metrics lie to us.

In [ ]:
predictions, targets = [x.numpy().flatten() for x in learn.get_preds(DatasetType.Valid)]
prediction_df = pd.DataFrame({'prediction': predictions, 'target': targets})

In [ ]:
(np.amin(predictions), np.amax(predictions))

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.tail()

### Calculate RMSE On Validation Set

Get the root mean squared error for the validation set only. This value we can compare against the public leaderboard on Kaggle, more or less.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
sqrt(mean_squared_error(prediction_df.target, prediction_df.prediction))

### Save The Model

If the model performed well, save it here (adding the validation error to the name) so that we can load it later on.

In [ ]:
learn.save('model_cv_3_885')

### Hyperparameter Testing

Hyperparameters are all those values that _we_ have to set to describe our model and how it learns (as opposed to the weights that are set _by the model_). For instance, we have to decide how many hidden layers it should have, and how many neurons there should be in each of those layers, etc.

Instead of using heuristics and experiments to find the optimal values for these, as we normally would, we can use Bayesian optimization in order to find them. I based the implementation on that of [this article](https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a).

**Feel free to skip this step!**

In [ ]:
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, plotting
import hyperopt.pyll.stochastic
import pickle

In [ ]:
def get_layers(params):
    '''Given a hyperparameter dictionary, return a pair of lists, one containing 
    the layer sizes and one the dropouts for each layer.'''
    layer_range = range(0, int(params['num_layers']))
    layers = [round(params['layer_size'] / pow(2, i)) for i in layer_range]
    ps = [params['layer_dropout'] / pow(10, i) for i in layer_range][::-1]
    return layers, ps

In [ ]:
def objective(params):
    '''Given a set of parameters, train the model for some epochs and return
    the lowest validation error found during those runs.'''
    layers, ps = get_layers(params)
    learn = tabular_learner(data,
                            layers=layers,
                            ps=ps,
                            emb_drop=params['emb_drop'],
                            y_range=y_range,
                            metrics=rmse)

    # Train for a set number of epochs.
    learn.fit_one_cycle(5, params['lr'], wd=params['wd'])

    print(f'Got losses: {learn.recorder.val_losses} for params {params}')
    
    # Extract the best score.
    best_score = min(learn.recorder.val_losses)
    
    # Return dictionary with information for evaluation.
    return {'loss': best_score, 'params': params, 'status': STATUS_OK}

Next we define the search space, iow the ranges of values within which we want the optimiser to look for solutions. For instance, we let it try models with anywhere between 1 and 6 hidden layers (the `num_layers` value).

In [ ]:
# Define the search space.
space = {
    'num_layers': hp.quniform('num_layers', 1, 6, 1),
    'layer_size': hp.quniform('layer_size', 25, 5000, 25),
    'layer_dropout': hp.loguniform('layer_dropout', np.log(1e-10), np.log(5e-1)),
    'emb_drop': hp.uniform('emb_drop', 1e-4, 5e-1),
    'wd': hp.uniform('wd', 0.1, 0.9),
    'lr': hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
}

Let's get a random sample from the space of possible sets of hyperparameters, to see that it makes sense.

In [ ]:
hyperopt.pyll.stochastic.sample(space)

Now we need to run the experiments to find the optimal hyperparameters. One experiment consists in training the model for a set number of epochs and then taking the lowest validation loss after any epoch. We can either do this starting from a previously saved state or from a clean slate.

The `max_evals` value states the maximum number of experiments to run. The optimiser will run either until the number has been reached or until it has stopped improving the parameters for a while.

In [ ]:
# Run this to load a previously saved set of parameter trials.
bayes_trials = pickle.load(open("hyperparam_trials.p", "rb"))

In [ ]:
# Run this to start afresh.
bayes_trials = Trials()

In [ ]:
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 25, trials = bayes_trials)

Now we can see which hyperparameters were used for the model that got the lowest loss. (Also use the `get_layers` function to see exactly how the layer sizes and dropouts were.)

In [ ]:
best

In [ ]:
get_layers(best)

In [ ]:
plotting.main_plot_history(bayes_trials)

In [ ]:
plotting.main_plot_histogram(bayes_trials)

In [ ]:
import pylab as pl
from hyperopt import base
pl.figure(figsize=(20, 10))
plotting.main_plot_vars(bayes_trials, colorize_best=1, columns=3, bandit=base.Domain(objective, space))

We can also save the trials made so far and continue running the optimizer later on.

In [ ]:
pickle.dump(bayes_trials, open("hyperparam_trials.p", "wb"))

## Make Submission Predictions

Finally, we need to run our model against the test set that is used by the competition's organizers to evaluate the competitors. We save the result to a `submission.csv` file which we'll then upload to Kaggle.

_Note: we should only do this at the very end, when we are happy with our hyperparameters. Otherwise, if we change our model based on our results on the public leaderboard, we risk overfitting our model to the 30% of samples used for the public leaderboard, and will fail to generalize for the remaining 70% of samples._

In [ ]:
out_df = test_df.copy(); out_df.head()

The test set has one row with some missing values (which we don't have in the training set), so let's use the most commonly occuring ones for that row.

In [ ]:
out_df.fillna(value={'first_active_monthYear': '2017.0',
                     'first_active_monthMonth': '12.0',
                     'first_active_monthWeek': '44.0'}, inplace=True)

In [ ]:
# Warning -- this takes quite a long time.
from tqdm import tqdm_notebook as tqdm
targets = []
for _, row in tqdm(out_df.iterrows()):
    targets.append(learn.predict(row)[2].numpy().flatten()[0])
out_df['target'] = pd.Series(targets)

In [ ]:
out_df['target'].to_csv('submission.csv.zip', header=['target'], index_label='card_id', compression='zip')